**Dataset:**
personality type data (Lab 2 - Personality Profile Type.csv)

**Objective:**
classify Personality type as one of the following **using RNNs**. <br>
['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']

**Evaluation metric:**
Precision

### Import used libraries

In [9]:
pip install contractions

In [10]:
pip install -U pip setuptools wheel


In [11]:
pip install -U spacy

In [12]:
! python -m spacy download en_core_web_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 29.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [13]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

In [14]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import re
import contractions
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from tqdm.auto import tqdm
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
import spacy
import nltk
from sklearn.preprocessing import LabelEncoder

In [15]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load Dataset

In [16]:
df = pd.read_csv ("Lab 2 - Personality Profile Type.csv")

In [17]:
df.head(10)

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments https://www.youtube.com/watch?v=iz7lE1g4XM4 sportscenter not top ten plays https://www.youtube.com/watch?v=uCdfze1etec pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8 http://www.youtube.com/watch?v=u8ejam5DP3E On repeat for most of today.|||May the PerC Experience immerse you.|||The last ...
1,ENTP,'I'm finding the lack of me in these posts very alarming.|||Sex can be boring if it's in the same position often. For example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary. There isn't enough...|||Giving new meaning to 'Game' theory.|||Hello *ENTP Grin* That's all it takes. Than we converse and they do most of the flirting while I acknowledge their presence and return their words with smooth wordplay and more cheeky grins.|||This...
2,INTP,"'Good one _____ https://www.youtube.com/watch?v=fHiGbolFFGw|||Of course, to which I say I know; that's my blessing and my curse.|||Does being absolutely positive that you and your best friend could be an amazing couple count? If so, than yes. Or it's more I could be madly in love in case I reconciled my feelings (which at...|||No, I didn't; thank you for a link!|||So-called Ti-Si loop (and it can stem from any current topic/obsession) can be deadly. It's like when you're stuck in your o..."
3,INTJ,"'Dear INTP, I enjoyed our conversation the other day. Esoteric gabbing about the nature of the universe and the idea that every rule and social code being arbitrary constructs created...|||Dear ENTJ sub, Long time no see. Sincerely, Alpha|||None of them. All other types hurt in deep existential ways that I want no part of.|||Probably a sliding scale that depends on individual preferences, like everything in humanity.|||Draco Malfoy also. I'd say he's either 358 or 368.|||I'm either 358..."
4,ENTJ,'You're fired.|||That's another silly misconception. That approaching is logically is going to be the key to unlocking whatever it is you think you are entitled to. Nobody wants to be approached with BS...|||But guys... he REALLY wants to go on a super-duper-long-ass vacation. C'mon guys. His boss just doesn't listen or get it. He even approached him logically and everything.|||Never mind. Just go on permanent vacation.|||Two months? I wouldn't be crazy about the idea. If you are really hi...
5,INTJ,"'18/37 @.@|||Science is not perfect. No scientist claims that it is, or that scientific information will not be revised as we discover new things. Rational thinking has been very useful to our society....|||INFP- Edgar Allen Poe was an INFP and he's in your siggy.|||People see the obvious Fi and are quick to put her as INFP. I agree that she has no Ne. I see her as an ISFP. Compare her to Haku (definite INFP). She is flat through most of Naruto.. but I don't...|||Lets get this party star..."
6,INFJ,"'No, I can't draw on my own nails (haha). Those were done by professionals on my nails. And yes, those are all gel. You mean those you posted were done by yourself on your own nails? Awesome!|||Probably the Electronic Screen Syndrome. With the advent of technology and social media, we all suffer from overstimulation on a daily basis. I'm guilty as well. In the past, I can be happy just...|||I love nail arts too! These are some of mine: 718282 718290 718298 718306 718314|||This is the first..."
7,INTJ,"'I tend to build up a collection of things on my desktop that i use frequently and then move them into a folder called 'Everything' from there it get sorted into type and sub type|||i ike to collect odd objects, even at work...a lot of people would call it junk but i like to collect it. Old unused software? ill take that off your hands :) i have a bunch of old adobe...|||i think its quite normal, i tend to only see my friends in real life every c

### Data splitting

In [18]:
#Extracting x, y from dataframe
x = df['posts']
y = df['type']

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify = y, random_state = 42)

In [20]:
# Concat x train and y train
train_data = pd.DataFrame(pd.concat([x_train, y_train], axis=1))


In [21]:
test_data = pd.DataFrame(pd.concat([x_test, y_test], axis=1))

In [22]:

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert the integer labels to categorical format
y_train_categorical = tf.keras.utils.to_categorical(y_train_encoded)
y_test_categorical = tf.keras.utils.to_categorical(y_test_encoded)


### EDA on training data

- check NaNs

In [23]:
pd.isnull(train_data).sum()

posts    0
type     0
dtype: int64

In [24]:
#No Null Values

- check duplicates

In [25]:
train_data.duplicated().sum()

0

- show a representative sample of data texts to find out required preprocessing steps

In [26]:
sample = train_data.sample(min(15, len(train_data)))
print("Sample Text Data:")
print(sample)

Sample Text Data:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    posts  \
1027  'I went to medical doctors for a health issue and felt this same way (I had a doctor admit that she googled a condition and then told me I might have it :happy:).   Sooo I guess my suggestion would...|||Jchazard Well, for the record, many of the most beautiful people I've ever known have been INFPs (yes, males included). I don't mean beautiful in a girly way, but just really great people.     I...|||lol ok..that's really letting someone off the hook. that's like 

- check dataset balancing

In [27]:
y_train.value_counts()/len(y_train)

type
INFP    0.211239
INFJ    0.169452
INTP    0.150288
INTJ    0.125793
ENTP    0.078963
ENFP    0.077810
ISTP    0.038905
ISFP    0.031268
ENTJ    0.026657
ISTJ    0.023631
ENFJ    0.021902
ISFJ    0.019164
ESTP    0.010231
ESFP    0.005476
ESFJ    0.004755
ESTJ    0.004467
Name: count, dtype: float64


 ♻  Cleaning and Preprocessing Steps are:  
 ---
  1.   Remove Links
  2.   Remove Emojis
  3.   Text Normalization
  4.   Remove newlines and tabs
  5.   Remove Extra Spaces
  6.   Remove contractions
  7.   Remove Punctuation
  8.   Remove Stopwords


### Cleaning and Preprocessing

In [28]:
posts_temporary = train_data['posts'].copy()

---
⚓ Remove User Names and Tags

In [29]:
def remove_usernames(text):
    # Remove user names starting with '@'
    return re.sub(r'@\w+\b', '', text)

In [30]:
def remove_tags(text):
    # Remove tags starting with '#'
    return re.sub(r'#\w+\b', '', text)

In [31]:
# Apply Remove user names
posts_temporary= posts_temporary.astype(str).apply(remove_usernames)
posts_temporary

8331    'this is actually exactly what i expected!  :laughing:  Introversion |||||||||| 40% |||||||||||||||| 70% Extroversion |||||||||||||||| 66% |||||||||| 36% Intuitive ||||||||||||||||||||...|||Now that you've had time to chill out, here comes the boundless insanity and awesomeness!  (p.s. just noticed the weird tense/perspective of this sentence... it's being spoken by the universe to me...|||I'm going to post before i actually read the entire thread:  Name a famous person you don't like. mel g...
1290    'Nope.  Not now, not ever.  I'm too busy with work, causes, and adrenaline-rush activities, and I'll do anything to make sure no one can keep up.  Freedom!  Forever!|||63913|||I did some thinking today and I decided to arrive at this conclusion.  I never really understood negative feelings in the past, so I tried to repress them hoping that they would go away and I would...|||How often do you guys find yourself miscommunicating with others, or using your own lingo when talking wi

In [32]:
# Apply Remove user names
posts_temporary= posts_temporary.astype(str).apply(remove_tags)
posts_temporary

8331    'this is actually exactly what i expected!  :laughing:  Introversion |||||||||| 40% |||||||||||||||| 70% Extroversion |||||||||||||||| 66% |||||||||| 36% Intuitive ||||||||||||||||||||...|||Now that you've had time to chill out, here comes the boundless insanity and awesomeness!  (p.s. just noticed the weird tense/perspective of this sentence... it's being spoken by the universe to me...|||I'm going to post before i actually read the entire thread:  Name a famous person you don't like. mel g...
1290    'Nope.  Not now, not ever.  I'm too busy with work, causes, and adrenaline-rush activities, and I'll do anything to make sure no one can keep up.  Freedom!  Forever!|||63913|||I did some thinking today and I decided to arrive at this conclusion.  I never really understood negative feelings in the past, so I tried to repress them hoping that they would go away and I would...|||How often do you guys find yourself miscommunicating with others, or using your own lingo when talking wi

---
⚓ REMOVE Links

In [33]:
def remove_links (text):
  preprocessed_text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)

  return preprocessed_text


In [34]:
posts_temporary= posts_temporary.astype(str).apply(remove_links)
posts_temporary

8331    'this is actually exactly what i expected!  :laughing:  Introversion |||||||||| 40% |||||||||||||||| 70% Extroversion |||||||||||||||| 66% |||||||||| 36% Intuitive ||||||||||||||||||||...|||Now that you've had time to chill out, here comes the boundless insanity and awesomeness!  (p.s. just noticed the weird tense/perspective of this sentence... it's being spoken by the universe to me...|||I'm going to post before i actually read the entire thread:  Name a famous person you don't like. mel g...
1290    'Nope.  Not now, not ever.  I'm too busy with work, causes, and adrenaline-rush activities, and I'll do anything to make sure no one can keep up.  Freedom!  Forever!|||63913|||I did some thinking today and I decided to arrive at this conclusion.  I never really understood negative feelings in the past, so I tried to repress them hoping that they would go away and I would...|||How often do you guys find yourself miscommunicating with others, or using your own lingo when talking wi

---
⚓ REMOVE Emojis

In [35]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001F004-\U0001F9CF"  # Miscellaneous Symbols and Pictographs
                           u"\U0001F600-\U0001F64F"  # Emoticons (iOS)
                           u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols (iOS)
                           u"\U0001F300-\U0001F5FF"  # Other additional symbols (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


In [36]:
posts_temporary= posts_temporary.astype(str).apply(remove_emoji)
posts_temporary

8331    'this is actually exactly what i expected!  :laughing:  Introversion |||||||||| 40% |||||||||||||||| 70% Extroversion |||||||||||||||| 66% |||||||||| 36% Intuitive ||||||||||||||||||||...|||Now that you've had time to chill out, here comes the boundless insanity and awesomeness!  (p.s. just noticed the weird tense/perspective of this sentence... it's being spoken by the universe to me...|||I'm going to post before i actually read the entire thread:  Name a famous person you don't like. mel g...
1290    'Nope.  Not now, not ever.  I'm too busy with work, causes, and adrenaline-rush activities, and I'll do anything to make sure no one can keep up.  Freedom!  Forever!|||63913|||I did some thinking today and I decided to arrive at this conclusion.  I never really understood negative feelings in the past, so I tried to repress them hoping that they would go away and I would...|||How often do you guys find yourself miscommunicating with others, or using your own lingo when talking wi

---
⚓ Text Normalization

In [37]:
#Lower casing
def lower_case(text):
  text_lowercase = text.str.lower()
  return text_lowercase

In [38]:
posts_temporary = lower_case(posts_temporary)
posts_temporary

8331    'this is actually exactly what i expected!  :laughing:  introversion |||||||||| 40% |||||||||||||||| 70% extroversion |||||||||||||||| 66% |||||||||| 36% intuitive ||||||||||||||||||||...|||now that you've had time to chill out, here comes the boundless insanity and awesomeness!  (p.s. just noticed the weird tense/perspective of this sentence... it's being spoken by the universe to me...|||i'm going to post before i actually read the entire thread:  name a famous person you don't like. mel g...
1290    'nope.  not now, not ever.  i'm too busy with work, causes, and adrenaline-rush activities, and i'll do anything to make sure no one can keep up.  freedom!  forever!|||63913|||i did some thinking today and i decided to arrive at this conclusion.  i never really understood negative feelings in the past, so i tried to repress them hoping that they would go away and i would...|||how often do you guys find yourself miscommunicating with others, or using your own lingo when talking wi

Note >> stemming is the next process of Normalization, it will be done after removing contractions

---
⚓ Remove Non-ASCI Charcters

In [39]:
# Remove all non-ASCII characters that still exists in the records

def remove_non_ascii(text):
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

In [40]:
posts_temporary= posts_temporary.astype(str).apply(remove_non_ascii)
posts_temporary

8331    'this is actually exactly what i expected!  :laughing:  introversion |||||||||| 40% |||||||||||||||| 70% extroversion |||||||||||||||| 66% |||||||||| 36% intuitive ||||||||||||||||||||...|||now that you've had time to chill out, here comes the boundless insanity and awesomeness!  (p.s. just noticed the weird tense/perspective of this sentence... it's being spoken by the universe to me...|||i'm going to post before i actually read the entire thread:  name a famous person you don't like. mel g...
1290    'nope.  not now, not ever.  i'm too busy with work, causes, and adrenaline-rush activities, and i'll do anything to make sure no one can keep up.  freedom!  forever!|||63913|||i did some thinking today and i decided to arrive at this conclusion.  i never really understood negative feelings in the past, so i tried to repress them hoping that they would go away and i would...|||how often do you guys find yourself miscommunicating with others, or using your own lingo when talking wi

---
⚓ Remove Lines & Tabs & Extra Spaces

In [41]:
def remove_lines_tabs_extraspaces(text):
    # Replace newlines and tabs with a single space
    text = re.sub(r'[\t\n\r]+', ' ', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [42]:
# Apply remove_lines_tabs_extraspaces
posts_temporary= posts_temporary.astype(str).apply(remove_lines_tabs_extraspaces)
posts_temporary

8331    'this is actually exactly what i expected! :laughing: introversion |||||||||| 40% |||||||||||||||| 70% extroversion |||||||||||||||| 66% |||||||||| 36% intuitive ||||||||||||||||||||...|||now that you've had time to chill out, here comes the boundless insanity and awesomeness! (p.s. just noticed the weird tense/perspective of this sentence... it's being spoken by the universe to me...|||i'm going to post before i actually read the entire thread: name a famous person you don't like. mel gibso...
1290    'nope. not now, not ever. i'm too busy with work, causes, and adrenaline-rush activities, and i'll do anything to make sure no one can keep up. freedom! forever!|||63913|||i did some thinking today and i decided to arrive at this conclusion. i never really understood negative feelings in the past, so i tried to repress them hoping that they would go away and i would...|||how often do you guys find yourself miscommunicating with others, or using your own lingo when talking with ot

---
⚓ Remove contractions

In [43]:
def remove_contractions(text):
    preprocessed_text = contractions.fix(text)
    return preprocessed_text


In [44]:
# Apply remove_contractions
posts_temporary= posts_temporary.apply(remove_contractions)
posts_temporary

8331    'this is actually exactly what i expected! :laughing: introversion |||||||||| 40% |||||||||||||||| 70% extroversion |||||||||||||||| 66% |||||||||| 36% intuitive ||||||||||||||||||||...|||now that you have had time to chill out, here comes the boundless insanity and awesomeness! (p.s. just noticed the weird tense/perspective of this sentence... it is being spoken by the universe to me...|||i am going to post before i actually read the entire thread: name a famous person you do not like. mel ...
1290    'nope. not now, not ever. i am too busy with work, causes, and adrenaline-rush activities, and i will do anything to make sure no one can keep up. freedom! forever!|||63913|||i did some thinking today and i decided to arrive at this conclusion. i never really understood negative feelings in the past, so i tried to repress them hoping that they would go away and i would...|||how often do you guys find yourself miscommunicating with others, or using your own lingo when talking with

---
⚓ Remove Punctuation

In [45]:
def remove_punctuation(text):
  output_text = text.translate(str.maketrans('', '', string.punctuation))
  return output_text

In [46]:
# Apply remove_punctuation
posts_temporary= posts_temporary.apply(remove_punctuation)
posts_temporary

8331    this is actually exactly what i expected laughing introversion  40  70 extroversion  66  36 intuitive now that you have had time to chill out here comes the boundless insanity and awesomeness ps just noticed the weird tenseperspective of this sentence it is being spoken by the universe to mei am going to post before i actually read the entire thread name a famous person you do not like mel gibson what was the last thing you bought groceries to make italian food name ai worked as a programmer...
1290    nope not now not ever i am too busy with work causes and adrenalinerush activities and i will do anything to make sure no one can keep up freedom forever63913i did some thinking today and i decided to arrive at this conclusion i never really understood negative feelings in the past so i tried to repress them hoping that they would go away and i wouldhow often do you guys find yourself miscommunicating with others or using your own lingo when talking with others i remember djarend

---
⚓ Remove Stop words

In [47]:
stop_words = set(stopwords.words('english'))

def remove_stop_words(text):
    words = word_tokenize(text)
    # Remove stop words
    filtered_words = [word for word in words if word.lower() not in stop_words]
    # Reconstruct the text without stop words
    return ' '.join(filtered_words)


In [48]:
posts_temporary= posts_temporary.apply(remove_stop_words)

In [49]:
sample = posts_temporary.sample(min(15, len(posts_temporary)))
print("Sample Text Data:")
print(sample)

Sample Text Data:
5759    well much relate post heck time high school years trying desperately someone else see young teen realized boys liked noti agree always sooo creative child grown creativity seems dwindling took art course college creativity time high imostly enfps met girl recently clicked instantlyshe positive sweet bubbly inspiring think enfpi really photography hubby bought digital slrbest gift ever expensive side gift relates girls interests never go wrong oh verywell actually get nagging feeling want exp...
5245    damn feel good actually let reformulate feel terrible drawing made day subway 612522 oh well loneliness probably worst feelingaw maybe regardless might hurt someone feelings well often fail realize make us happy fame money things often seek yet true happiness isyou sweet 3i miss faith humankindat times hang around interwebs discuss people usually guys meet irl lose interest friend boyfriend happened friendi relate way much today life sucks sometimes got agree hu

In [50]:
x_train = posts_temporary


#**⚓ Same preprocessing for test data**

In [51]:
posts_temporary_test = test_data['posts'].copy()

In [52]:
# Apply Remove user names
posts_temporary_test= posts_temporary_test.astype(str).apply(remove_usernames)
posts_temporary_test

7814    'Macona , it depends if the big family has extroverted people or not. Everyone literally in my house is an introvert (2 ISTJs, INFJ, IXXX, INFJ/INFP, INTJ, and the baby of my family -- my brother ---...|||^ not to break the infp erotica theme going here, but i was just on fb and saw this acquaintence/friend i thought was interesting and kinda dateable was in a relationship with a guy. man, i have...|||exactly me! though i tend to turn to god a lot even when i'm not feeling like anything in p...
2233    https://www.youtube.com/watch?v=cfg4SC9Nics|||https://www.youtube.com/watch?v=IMq979eouf8|||Blodsmak - Sveltihel|||https://www.youtube.com/watch?v=IN1skVSX8mM|||https://www.youtube.com/watch?v=r4lDJ0geURY|||He's brilliant! But this was the episode where he had Regeneration Sickness. So we should wait on judging before we see more of him :P|||https://www.youtube.com/watch?v=Yp5NHqyljzs|||https://www.youtube.com/watch?v=eSOHOMKWJEg|||That's kinda cool... It's somehow modern with an

In [53]:
# Apply Remove user names
posts_temporary= posts_temporary.astype(str).apply(remove_tags)
posts_temporary

8331    actually exactly expected laughing introversion 40 70 extroversion 66 36 intuitive time chill comes boundless insanity awesomeness ps noticed weird tenseperspective sentence spoken universe mei going post actually read entire thread name famous person like mel gibson last thing bought groceries make italian food name ai worked programmer high school went college comp sci year eventually decided something enjoyed something wanted alli buy whenever possible adjust cooking habits accommodate lo...
1290    nope ever busy work causes adrenalinerush activities anything make sure one keep freedom forever63913i thinking today decided arrive conclusion never really understood negative feelings past tried repress hoping would go away wouldhow often guys find miscommunicating others using lingo talking others remember djarendees post back said speaking like20 would cut worms see would happen heck kind question thati think chessboxing would totally awesome sport istpi lumbar puncture clini

In [54]:
posts_temporary_test= posts_temporary_test.astype(str).apply(remove_links)
posts_temporary_test

7814    'Macona , it depends if the big family has extroverted people or not. Everyone literally in my house is an introvert (2 ISTJs, INFJ, IXXX, INFJ/INFP, INTJ, and the baby of my family -- my brother ---...|||^ not to break the infp erotica theme going here, but i was just on fb and saw this acquaintence/friend i thought was interesting and kinda dateable was in a relationship with a guy. man, i have...|||exactly me! though i tend to turn to god a lot even when i'm not feeling like anything in p...
2233                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [55]:
posts_temporary_test= posts_temporary_test.astype(str).apply(remove_emoji)
posts_temporary_test

7814    'Macona , it depends if the big family has extroverted people or not. Everyone literally in my house is an introvert (2 ISTJs, INFJ, IXXX, INFJ/INFP, INTJ, and the baby of my family -- my brother ---...|||^ not to break the infp erotica theme going here, but i was just on fb and saw this acquaintence/friend i thought was interesting and kinda dateable was in a relationship with a guy. man, i have...|||exactly me! though i tend to turn to god a lot even when i'm not feeling like anything in p...
2233                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [56]:
posts_temporary_test = lower_case(posts_temporary_test)
posts_temporary_test

7814    'macona , it depends if the big family has extroverted people or not. everyone literally in my house is an introvert (2 istjs, infj, ixxx, infj/infp, intj, and the baby of my family -- my brother ---...|||^ not to break the infp erotica theme going here, but i was just on fb and saw this acquaintence/friend i thought was interesting and kinda dateable was in a relationship with a guy. man, i have...|||exactly me! though i tend to turn to god a lot even when i'm not feeling like anything in p...
2233                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [57]:
#remove_non_ascii
posts_temporary_test= posts_temporary_test.astype(str).apply(remove_non_ascii)
posts_temporary_test

7814    'macona , it depends if the big family has extroverted people or not. everyone literally in my house is an introvert (2 istjs, infj, ixxx, infj/infp, intj, and the baby of my family -- my brother ---...|||^ not to break the infp erotica theme going here, but i was just on fb and saw this acquaintence/friend i thought was interesting and kinda dateable was in a relationship with a guy. man, i have...|||exactly me! though i tend to turn to god a lot even when i'm not feeling like anything in p...
2233                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [58]:
# Apply remove_lines_tabs_extraspaces
posts_temporary_test= posts_temporary_test.astype(str).apply(remove_lines_tabs_extraspaces)
posts_temporary_test

7814    'macona , it depends if the big family has extroverted people or not. everyone literally in my house is an introvert (2 istjs, infj, ixxx, infj/infp, intj, and the baby of my family -- my brother ---...|||^ not to break the infp erotica theme going here, but i was just on fb and saw this acquaintence/friend i thought was interesting and kinda dateable was in a relationship with a guy. man, i have...|||exactly me! though i tend to turn to god a lot even when i'm not feeling like anything in p...
2233                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [59]:
# Apply remove_contractions
posts_temporary_test= posts_temporary_test.apply(remove_contractions)
posts_temporary_test

7814    'macona , it depends if the big family has extroverted people or not. everyone literally in my house is an introvert (2 istjs, infj, ixxx, infj/infp, intj, and the baby of my family -- my brother ---...|||^ not to break the infp erotica theme going here, but i was just on fb and saw this acquaintence/friend i thought was interesting and kind of dateable was in a relationship with a guy. man, i have...|||exactly me! though i tend to turn to god a lot even when i am not feeling like anything i...
2233                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [60]:
# Apply remove_punctuation
posts_temporary_test= posts_temporary_test.apply(remove_punctuation)
posts_temporary_test

7814    macona  it depends if the big family has extroverted people or not everyone literally in my house is an introvert 2 istjs infj ixxx infjinfp intj and the baby of my family  my brother  not to break the infp erotica theme going here but i was just on fb and saw this acquaintencefriend i thought was interesting and kind of dateable was in a relationship with a guy man i haveexactly me though i tend to turn to god a lot even when i am not feeling like anything in particular i feel like god is m...
2233                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [61]:
posts_temporary_test= posts_temporary_test.apply(remove_stop_words)

In [62]:
posts_temporary_test

7814    macona depends big family extroverted people everyone literally house introvert 2 istjs infj ixxx infjinfp intj baby family brother break infp erotica theme going fb saw acquaintencefriend thought interesting kind dateable relationship guy man haveexactly though tend turn god lot even feeling like anything particular feel like god family like watchful parent distraught feltwoah mine dads istj moms isfj siblings istj ixxx seriously tried type think psychopath typed intji mia site couple days ...
2233                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [63]:
x_test  = posts_temporary_test

# ⚓ ***#Word Embedding***

In [64]:
from keras.preprocessing.text import Tokenizer

In [65]:
texts = x_train.tolist()  # Convert Series to list

In [66]:
tokenizer = Tokenizer()

In [67]:
tokenizer.fit_on_texts(texts)

In [68]:
word2idx = tokenizer.word_index

In [69]:
print(word2idx)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [70]:
idx2word = {word2idx[word]: word for word in word2idx.keys()}

In [71]:
idx2word[194]

'relate'

In [72]:
def reconstruct(tokens):
    text = []
    for token in tokens:
        text.append(idx2word[token])
    return " ".join(text)

In [73]:
reconstruct (idx2word)

'like would think people know really one get time feel much say things want love way see good something also could lot always type someone even well make never going find go life thing though friends actually sure person pretty right need first infp kind still work got infj thought said intj many us friend take read mean try around probably best years sometimes back intp little anything maybe thread different usually understand long others better seems might post agree two makes enfp look may hard yes trying everyone thinking since types point bit quite ever guess tell come 2 talk world relationship believe seem day made definitely let mind feeling use new school give every great often help entp guys used bad everything personality question interesting times tend part anyone last enough sense idea talking another keep mbti least hate getting 3 else true put care problem 1 experience found without however either year guy reading especially thank reason wrong sounds stuff nothing feeling

### Let' pad x_train to make sure they are all of the same length

In [74]:
x_train_sequences = tokenizer.texts_to_sequences(x_train)


In [75]:
# with respect to tokens not sentence length
max_sequence_len = 0
for sentence in x_train:
    max_sequence_len = max(len(sentence), max_sequence_len)
print(max_sequence_len)

6738


Because this max length is too much, let's make it up to 100

In [76]:
max_sequence_len = 100

Now let's padd all the sentences to have that max length

In [77]:
x_train_padded = np.zeros((x_train.shape[0], max_sequence_len))

In [78]:
for i, sent in enumerate(x_train_sequences):
    sentence_length = min(len(sent), max_sequence_len)
    x_train_padded[i, :sentence_length] = sent[:sentence_length]

In [79]:
x_train_padded.shape

(6940, 100)

the same for x_test

In [80]:
x_test_sequences = tokenizer.texts_to_sequences(x_test)


In [81]:
# with respect to tokens not sentence length
max_sequence_len_test = 0
for sentence in x_test:
    max_sequence_len_test = max(len(sentence), max_sequence_len)
print(max_sequence_len_test)

3441


In [82]:
max_sequence_len_test = 100

In [83]:
x_test_padded = np.zeros((len(x_test_sequences), max_sequence_len_test))
for i, sent in enumerate(x_test_sequences):
    sentence_length = min(len(sent), max_sequence_len_test)
    x_test_padded[i, :sentence_length] = sent[:sentence_length]

**You  are doing Great so far!**

### Modelling

In [84]:
x_train_padded

array([[3.7000e+01, 1.8800e+02, 1.4070e+03, ..., 2.2720e+03, 2.2720e+03,
        2.7200e+02],
       [1.9660e+03, 9.6000e+01, 9.1800e+02, ..., 6.0400e+03, 3.6920e+03,
        1.0000e+00],
       [8.7000e+01, 1.3330e+03, 5.1220e+03, ..., 3.4027e+04, 7.2100e+02,
        3.9450e+03],
       ...,
       [9.2600e+02, 2.1600e+02, 1.1690e+03, ..., 2.7890e+03, 2.1200e+02,
        5.2400e+02],
       [4.0000e+00, 1.7510e+03, 3.9980e+03, ..., 3.4000e+01, 4.2000e+01,
        1.7200e+02],
       [1.8500e+03, 4.1000e+01, 8.0000e+01, ..., 2.2000e+01, 4.0400e+02,
        1.6000e+01]])

In [85]:
y_train_categorical

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [86]:
x_test_padded

array([[3.5900e+02, 2.0200e+02, 2.2500e+02, ..., 1.1980e+03, 4.0000e+00,
        2.6500e+02],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.6700e+02, 1.4690e+03, 1.7150e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [2.5500e+02, 4.6400e+02, 5.6300e+02, ..., 6.0000e+02, 2.4000e+01,
        5.3600e+02],
       [8.0000e+00, 1.2310e+03, 1.0300e+02, ..., 3.1700e+02, 9.1000e+01,
        6.7000e+01],
       [7.8670e+03, 1.5000e+01, 6.6900e+02, ..., 1.0535e+04, 2.0290e+04,
        9.0000e+00]])

In [87]:
y_test_categorical

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [88]:
VOCAB_SIZE = len(word2idx)


In [117]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 64),
    tf.keras.layers.SimpleRNN(64),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation='softmax')
])

In [118]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'Precision'], run_eagerly=True)


In [119]:
History  = model.fit (
    x_train_padded,
    y_train_categorical,
    epochs=5,
    batch_size=128,
)

Epoch 1/5
55/55 [==============================] - 24s 432ms/step - loss: 2.4950 - accuracy: 0.1715 - precision_3: 0.0000e+00
Epoch 2/5
55/55 [==============================] - 24s 444ms/step - loss: 2.2881 - accuracy: 0.2228 - precision_3: 0.5000
Epoch 3/5
55/55 [==============================] - 23s 409ms/step - loss: 1.9869 - accuracy: 0.3782 - precision_3: 0.8310
Epoch 4/5
55/55 [==============================] - 21s 382ms/step - loss: 1.4473 - accuracy: 0.5762 - precision_3: 0.8964
Epoch 5/5
55/55 [==============================] - 21s 390ms/step - loss: 0.9852 - accuracy: 0.7107 - precision_3: 0.9320


#### Evaluation

**Evaluation metric:**
Precision

In [120]:
model.evaluate(x_test_padded, y_test_categorical)

55/55 [==============================] - 10s 184ms/step - loss: 2.8722 - accuracy: 0.1504 - precision_3: 0.1943


[2.872217893600464, 0.1504322737455368, 0.19426049292087555]

### Enhancement

### Conclusion and final results


#### Done!